# 1. Data preprocessing

In [5]:
import joblib
import pandas as pd
import numpy as np

In [7]:
players = joblib.load("chgk/players.pkl")
results = joblib.load("chgk/results.pkl")
tournaments = joblib.load("chgk/tournaments.pkl")

In [8]:
len(tournaments)

5528

In [9]:
def dict_to_df(data):
    if isinstance(data, dict):
        return pd.DataFrame(data.values())
    else:
        return pd.DataFrame(data)

def save_to_paruet(df, filename):
    df.to_parquet(filename)
    
def load_from_parquet(filename):
    return pd.read_parquet(filename)

In [10]:
df_players = dict_to_df(players)
df_results = dict_to_df(
    [team.update({"tournament_id": tid}) or team for tid, teams in results.items() for team in teams]
)
df_tournaments = dict_to_df(tournaments)

In [41]:
df_players_norm = df_players.copy()

In [11]:
df_tournaments_norm = df_tournaments.copy()

df_tournaments_norm["dateStart"] = pd.to_datetime(df_tournaments_norm["dateStart"])
df_tournaments_norm["dateEnd"] = pd.to_datetime(df_tournaments_norm["dateEnd"])
df_tournaments_norm["type"] = df_tournaments_norm["type"].apply(lambda t: t["name"])
df_tournaments_norm["season"] = df_tournaments_norm["season"].apply(lambda s: s and int(s.split("/")[2]))

df_tournaments_norm = df_tournaments_norm.drop(columns=["orgcommittee", "synchData"])
save_to_paruet(df_tournaments_norm, "chgk/df_tournaments_norm.parquet")

df_tournaments_norm

,id,name,dateStart,dateEnd,type,season,questionQty
0,1,Чемпионат Южного Кавказа,2003-07-25 00:00:00+04:00,2003-07-27 00:00:00+04:00,Обычный,1.0,None
1,2,Летние зори,2003-08-09 00:00:00+04:00,2003-08-09 00:00:00+04:00,Обычный,1.0,None
2,3,Турнир в Ижевске,2003-11-22 00:00:00+03:00,2003-11-24 00:00:00+03:00,Обычный,2.0,None
3,4,Чемпионат Украины. Переходной этап,2003-10-11 00:00:00+04:00,2003-10-12 00:00:00+04:00,Обычный,2.0,None
4,5,Бостонское чаепитие,2003-10-10 00:00:00+04:00,2003-10-13 00:00:00+04:00,Обычный,2.0,None
...,...,...,...,...,...,...,...
5523,6481,Онлайн: 15:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-05 15:00:00+03:00,2020-05-05 18:00:00+03:00,Обычный,53.0,"{'1': 12, '2': 12, '3': 12}"
5524,6482,Онлайн: 19:00 Зелёный шум,2020-05-07 19:00:00+03:00,2020-05-07 21:30:00+03:00,Обычный,53.0,"{'1': 13, '2': 13, '3': 13}"
5525,6483,Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-08 19:00:00+03:00,2020-05-08 21:30:00+03:00,Обычный,53.0,"{'1': 12, '2': 12, '3': 12}"
5526,6484,"Онлайн: 22:00 Не числом, а умением - 2 (NEW!)",2020-05-04 22:00:00+03:00,2020-05-04 23:40:00+03:00,Обычный,53.0,"{'1': 12, '2': 12}"


In [82]:
df_results_norm = df_results.copy()
df_results_norm["team_id"] = df_results_norm["team"].apply(lambda t: t["id"])
df_results_norm["team_name"] = df_results_norm["team"].apply(lambda t: t["name"])
df_results_norm["flags"] = df_results_norm["flags"].apply(lambda flags: list(map(lambda f: f["longName"], flags)))
df_results_norm = df_results_norm.drop(columns=["team", "current", "synchRequest", "controversials"])

# df_results_norm = df_results_norm.explode("teamMembers")
df_results_norm["teamMembers"] = df_results_norm["teamMembers"].apply(lambda ps: list(map(lambda p: p["player"]["id"], ps)))

# df_results_norm = df_results_norm[df_results_norm["teamMembers"].isna() == False]
# df_results_norm["player_id"] = df_results_norm["teamMembers"].apply(lambda p: p["player"]["id"])
# df_results_norm["player_name"] = df_results_norm["teamMembers"].apply(lambda p: p["player"]["name"])
# df_results_norm["player_surname"] = df_results_norm["teamMembers"].apply(lambda p: p["player"]["surname"])
# df_results_norm["player_flag"] = df_results_norm["teamMembers"].apply(lambda p: p["flag"])
# df_results_norm["player_usedRating"] = df_results_norm["teamMembers"].apply(lambda p: p["usedRating"])
# df_results_norm["player_rating"] = df_results_norm["teamMembers"].apply(lambda p: p["rating"])

df_results_norm = df_results_norm[df_results_norm["mask"].isna() == False]
df_results_norm["questions_count"] = df_results_norm["mask"].apply(lambda mask: len(mask))
df_results_norm["answered"] = df_results_norm["questionsTotal"] / df_results_norm["questions_count"]
df_results_norm = df_results_norm.rename(columns={"teamMembers": "team_members", "questionsTotal": "questions_total"})

df_results_norm

,mask,questions_total,position,flags,team_members,tournament_id,team_id,team_name,questions_count,answered
2407,0111011101101110001101110011111111110011111100...,67.0,1.0,[],"[1560, 2935, 3270, 4878, 18935, 32979, 36497]",22,1,Неспроста,90,0.744444
2408,0111111101011010010101110111111111110011011111...,65.0,2.5,[],"[707, 13551, 15442, 25882, 30475, 34846]",22,2,Афина,90,0.722222
2409,0011111101011010011101110011111111110111111111...,65.0,2.5,[],"[2694, 20691, 22482, 22935, 28513, 29800]",22,670,Ксеп,90,0.722222
2410,0111111001101110001101111011111111110100111111...,64.0,4.5,[],"[5526, 14259, 18686, 20260, 21805, 29333]",22,173,ЮМА,90,0.711111
2411,0111111001101111001111111011111110110010111110...,64.0,4.5,[],[],22,175,Транссфера,90,0.711111
...,...,...,...,...,...,...,...,...,...,...
528738,100110001100111011011110100100010101010,20.0,3.0,[],"[138479, 80384, 133207, 95145, 121594, 133209]",6456,55612,Экстремаль,39,0.512821
528739,100110101100001111110110100100010001010,19.0,4.5,[],[64129],6456,68457,Орда,39,0.487179
528740,000001111100111001010110000100111001011,19.0,4.5,[],"[111255, 166584, 135103, 103290, 103293]",6456,43261,Господин Брейтенбихер,39,0.487179
528741,100101101100100100010110100100010001010,16.0,6.0,[],"[96552, 192899, 192900, 123238, 129706, 192901]",6456,69918,Teddyhaters,39,0.410256


In [83]:
df_players_norm.to_csv("chgk/players.csv", index=False)
df_results_norm.to_csv("chgk/results.csv", index=False)
df_tournaments_norm.to_csv("chgk/tournaments.csv", index=False)

,id,name,dateStart,dateEnd,type,season,questionQty
0,1,Чемпионат Южного Кавказа,2003-07-25 00:00:00+04:00,2003-07-27 00:00:00+04:00,Обычный,1.0,NaN
1,2,Летние зори,2003-08-09 00:00:00+04:00,2003-08-09 00:00:00+04:00,Обычный,1.0,NaN
2,3,Турнир в Ижевске,2003-11-22 00:00:00+03:00,2003-11-24 00:00:00+03:00,Обычный,2.0,NaN
3,4,Чемпионат Украины. Переходной этап,2003-10-11 00:00:00+04:00,2003-10-12 00:00:00+04:00,Обычный,2.0,NaN
4,5,Бостонское чаепитие,2003-10-10 00:00:00+04:00,2003-10-13 00:00:00+04:00,Обычный,2.0,NaN
...,...,...,...,...,...,...,...
5523,6481,Онлайн: 15:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-05 15:00:00+03:00,2020-05-05 18:00:00+03:00,Обычный,53.0,"{'1': 12, '2': 12, '3': 12}"
5524,6482,Онлайн: 19:00 Зелёный шум,2020-05-07 19:00:00+03:00,2020-05-07 21:30:00+03:00,Обычный,53.0,"{'1': 13, '2': 13, '3': 13}"
5525,6483,Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-08 19:00:00+03:00,2020-05-08 21:30:00+03:00,Обычный,53.0,"{'1': 12, '2': 12, '3': 12}"
5526,6484,"Онлайн: 22:00 Не числом, а умением - 2 (NEW!)",2020-05-04 22:00:00+03:00,2020-05-04 23:40:00+03:00,Обычный,53.0,"{'1': 12, '2': 12}"


In [264]:
df_players_norm = pd.read_csv("chgk/players.csv")
df_results_norm = pd.read_csv("chgk/results.csv")
df_tournaments_norm = pd.read_csv("chgk/tournaments.csv")

In [84]:
df = df_results_norm.merge(df_tournaments_norm, how="left", left_on="tournament_id", right_on="id")
df["dateStart"] = pd.to_datetime(df["dateStart"], utc=True)
df_train = df[df["dateStart"].dt.year == 2019]
df_test = df[df["dateStart"].dt.year == 2020]

df_train.to_csv("chgk/train.csv", index=False)
df_test.to_csv("chgk/test.csv", index=False)

# 2. Baseline model

In [126]:
import ast

from scipy.sparse import lil_matrix, csr_matrix
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [6]:
df_train = pd.read_csv("chgk/train.csv", parse_dates=["dateStart", "dateEnd"])
df_test = pd.read_csv("chgk/test.csv", parse_dates=["dateStart", "dateEnd"])

# Hacky & potentially dangeourus
df_train["team_members"] = df_train["team_members"].apply(ast.literal_eval)
df_test["team_members"] = df_test["team_members"].apply(ast.literal_eval)

df_train

,mask,questions_total,position,flags,team_members,tournament_id,team_id,team_name,questions_count,answered,id,name,dateStart,dateEnd,type,season,questionQty
0,111111111011111110111111111100010010,28.0,1.0,[],"[6212, 18332, 18036, 22799, 15456, 26089]",4772,45556,Рабочее название,36,0.777778,4772,Синхрон северных стран. Зимний выпуск,2019-01-05 16:00:00+00:00,2019-01-09 19:00:00+03:00,Синхрон,52.0,"{'1': 12, '2': 12, '3': 12}"
1,111111111011110100101111011001011010,25.0,5.5,[],"[1585, 40840, 1584, 10998, 16206]",4772,1030,Сборная Бутана,36,0.694444,4772,Синхрон северных стран. Зимний выпуск,2019-01-05 16:00:00+00:00,2019-01-09 19:00:00+03:00,Синхрон,52.0,"{'1': 12, '2': 12, '3': 12}"
2,111111111011110101101111001011110000,25.0,5.5,[],"[23513, 18168, 21060, 35850, 31332, 10187]",4772,4252,Ять,36,0.694444,4772,Синхрон северных стран. Зимний выпуск,2019-01-05 16:00:00+00:00,2019-01-09 19:00:00+03:00,Синхрон,52.0,"{'1': 12, '2': 12, '3': 12}"
3,101111101111111110001101011001111010,25.0,5.5,[],"[36742, 28939, 54289, 15381, 27375]",4772,5444,Эйфью,36,0.694444,4772,Синхрон северных стран. Зимний выпуск,2019-01-05 16:00:00+00:00,2019-01-09 19:00:00+03:00,Синхрон,52.0,"{'1': 12, '2': 12, '3': 12}"
4,111111101011111101000111001001111110,25.0,5.5,[],"[28689, 17720, 30597, 12400, 26988, 69476]",4772,40931,Здоровенный Я,36,0.694444,4772,Синхрон северных стран. Зимний выпуск,2019-01-05 16:00:00+00:00,2019-01-09 19:00:00+03:00,Синхрон,52.0,"{'1': 12, '2': 12, '3': 12}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87068,000000000000000010000000000000000000,1.0,2013.0,[],"[215629, 215630, 215631]",6255,75837,Штиль,36,0.027778,6255,ОВСЧ,2019-09-20 17:00:00+00:00,2020-02-19 23:59:00+03:00,Общий зачёт,53.0,"{'1': 36, '2': 36, '3': 36, '4': 36, '5': 36, ..."
87069,000000000000100000000000000000000000,1.0,2013.0,[],"[217541, 217542, 217543]",6255,76236,Булачёв и ко,36,0.027778,6255,ОВСЧ,2019-09-20 17:00:00+00:00,2020-02-19 23:59:00+03:00,Общий зачёт,53.0,"{'1': 36, '2': 36, '3': 36, '4': 36, '5': 36, ..."
87070,0X0000100000000000000000000000000000,1.0,2013.0,[],"[221510, 221511, 221512, 221513, 221514, 221515]",6255,77170,Сонное царство,36,0.027778,6255,ОВСЧ,2019-09-20 17:00:00+00:00,2020-02-19 23:59:00+03:00,Общий зачёт,53.0,"{'1': 36, '2': 36, '3': 36, '4': 36, '5': 36, ..."
87071,0X0000000000000000000000100000000000,1.0,2013.0,[],"[220732, 221563, 221564, 221565, 221566, 221567]",6255,77187,Новички,36,0.027778,6255,ОВСЧ,2019-09-20 17:00:00+00:00,2020-02-19 23:59:00+03:00,Общий зачёт,53.0,"{'1': 36, '2': 36, '3': 36, '4': 36, '5': 36, ..."


In [7]:
df_train["questions_count"].value_counts()[0:5]

36    64687
45     6123
48     3012
39     2075
72     1649
Name: questions_count, dtype: int64

In [57]:
TOUR_QUESTIONS_COUNT = 36

df_train = df_train[df_train["questions_count"] == TOUR_QUESTIONS_COUNT]
df_test = df_test[df_test["questions_count"] == TOUR_QUESTIONS_COUNT]

df_train.shape, df_test.shape

((64687, 17), (16912, 17))

In [51]:
MEMBERS_COUNT = np.sum(df_train["team_members"].apply(len))
MEMBERS_COUNT

325600

In [54]:
tournament_ids = list(set(df_train["tournament_id"]))
tournament_id_to_idx = {id:i for i, id in enumerate(tournament_ids)}

In [55]:
def to_flatten_set(series):
    res = set()
    for line in series.tolist():
        for item in line:
            res.add(item)
    return res

train_players = to_flatten_set(df_train["team_members"])

player_idx_to_id = list(train_players)
player_id_to_idx = {id:idx for idx, id in enumerate(player_idx_to_id)}

In [16]:
def player_id_to_idx_func(id):
    return player_id_to_idx[id]

def player_ids_to_idx_func(ids):
    return list(map(player_id_to_idx_func, ids))

def player_idxs_to_id_func(idxs):
    return list(map(lambda idx: player_idx_to_id[idx], idxs))

PLAYERS_COUNT = len(player_id_to_idx)
PLAYERS_COUNT

52487

Let X be concatenation of one-hot encoded player id concatenated with one-hot encoded question id

An Y - 1 if question was answered, else 0


In [242]:
def to_player_matrix(df):
    x_matrix = lil_matrix(
        (
            MEMBERS_COUNT * TOUR_QUESTIONS_COUNT, # rows
            PLAYERS_COUNT + (TOUR_QUESTIONS_COUNT * len(tournament_ids)), # cols
        ),
        dtype=np.int8,
    )
    y = np.zeros(MEMBERS_COUNT * TOUR_QUESTIONS_COUNT)

    # Array to store size of teams to estimate relatve power of memebers later in EM-algo
    members_count_seq = []
    ii = 0
        
    for _, row in tqdm(df.iterrows(), total=len(df)):
        tour_offset = PLAYERS_COUNT + (tournament_id_to_idx[row["tournament_id"]] * TOUR_QUESTIONS_COUNT)
        player_indexes = player_ids_to_idx_func(row["team_members"])
        members_count_seq.append(len(player_indexes))
        
        for qi, qa in enumerate(row["mask"]):
            for player_idx in player_indexes:            
                x_matrix[ii, player_idx] = 1
                x_matrix[ii, tour_offset + qi] = 1
            
                if qa == "1":
                    y[ii] = 1
                
                ii += 1
    
    # lil_matrix type is good to build sparse matrixes
    # for calculations convert it to csr_matrix
    return csr_matrix(x_matrix), y, np.array(members_count_seq)

players_train_mtx, train_y, members_count_seq = to_player_matrix(df_train)
players_train_mtx.shape, train_y.shape, members_count_seq.shape

100%|██████████| 64687/64687 [03:08<00:00, 343.50it/s]


((11721600, 66815), (11721600,), (64687,))

In [243]:
lr_model = LinearRegression()
lr_model.fit(players_train_mtx, train_y)

LinearRegression()

In [22]:
# Download real players rating from https://rating.chgk.info/players.php in CSV format and compare

df_real_top = pd.read_csv("chgk/players-release-2021-03-26.csv")
df_real_top = df_real_top.rename(columns={" ИД": "id", "Место": "real_place"})[["id", "real_place"]]
df_real_top

,id,real_place
0,30152,1
1,27822,2
2,28751,3
3,30270,4
4,27403,5
...,...,...
38355,223668,38249
38356,87713,99999
38357,113716,99999
38358,141296,99999


In [245]:
# Coeff inside our linear model for each player should show the player impact
# So we can use these coeffs to compare players with each other

df_lr_rank = pd.DataFrame(
    [(coeff, player_idx_to_id[i]) for i, coeff in enumerate(lr_model.coef_[0:PLAYERS_COUNT])],
    columns=["rank", "id"],
)

# Match real top with our predicted top
df_lr_rank = df_players_norm.merge(df_lr_rank, how="inner", on="id").sort_values("rank", ascending=False).drop(columns=["patronymic"])
df_lr_rank = df_lr_rank.merge(df_real_top, how="left", on="id") 
                          
df_lr_rank.head(50)

,id,name,surname,rank,real_place
0,36844,Павел,Щербина,0.687205,13833.0
1,27403,Максим,Руссо,0.662136,5.0
2,26798,Анна,Резникова,0.647536,5816.0
3,38175,Максим,Пилипенко,0.646817,9962.0
4,27822,Михаил,Савченков,0.645153,2.0
5,28751,Иван,Семушин,0.642976,3.0
6,4270,Александра,Брутер,0.636396,6.0
7,6000,Игорь,Вольфсон,0.635845,10964.0
8,30270,Сергей,Спешков,0.632697,4.0
9,30152,Артём,Сорожкин,0.623911,1.0


In [246]:
player_to_predicted_rank = df_lr_rank.set_index("id")["rank"].to_dict()

# 3. Evaluate baseline model

In [247]:
from scipy.stats import spearmanr, kendalltau

In [248]:
df_test_known = df_test.copy()[["position", "questions_total", "team_members", "tournament_id", "team_id", "answered"]]
df_test_known["team_members"] = df_test_known["team_members"].apply(lambda t: [id for id in t if id in train_players])
df_test_known = df_test_known[df_test_known["team_members"].apply(lambda t: len(t) > 0)]

df_test_known["pred_rating"] = df_test_known["team_members"].apply(
    lambda t: -np.sum([player_to_predicted_rank[id] for id in t])
)

df_test_known=df_test_known.sort_values(["tournament_id", "team_id"])

df_test_known

,position,questions_total,team_members,tournament_id,team_id,answered,pred_rating
94,2.5,31.0,"[6482, 32458, 36120, 25882, 34846, 30475]",5414,2,0.861111,-3.346489
216,124.0,21.0,"[3800, 9707, 11877, 61401, 33155, 12409]",5414,47,0.583333,-2.256905
315,220.5,15.0,"[8855, 28453, 193755, 11247, 139433]",5414,87,0.416667,-1.149064
157,66.5,24.0,"[3207, 33768, 35120, 22007, 22006, 16550]",5414,167,0.666667,-2.394676
100,8.0,29.0,"[11466, 5056, 4223, 18277, 21931, 30306]",5414,216,0.805556,-2.667245
...,...,...,...,...,...,...,...
21764,3.0,16.0,"[14159, 36817, 29671, 24069]",6410,2111,0.444444,-1.521059
21762,1.0,21.0,"[31999, 67805, 21337, 70888]",6410,55283,0.583333,-1.682965
21765,4.0,2.0,"[133442, 179291, 161570, 197302]",6410,60732,0.055556,0.098589
21766,5.0,0.0,"[217110, 217112]",6410,63157,0.000000,0.515461


In [286]:
def score_result_df(df_test_known):
    df_test_known = df_test_known.sort_values(["tournament_id", "team_id"])

    baseline_true = []
    baseline_preds = []

    for tournament_id in set(df_test_known.tournament_id):
        baseline_true.append(df_test_known[df_test_known["tournament_id"] == tournament_id]["position"].tolist())
        baseline_preds.append(df_test_known[df_test_known["tournament_id"] == tournament_id]["pred_rating"].tolist())
        
    spearmanr_res = []
    kendalltau_res = []

    for bt, bp in zip(baseline_true, baseline_preds):
        corr, _ = spearmanr(bt, bp)
        if not np.isnan(corr):
            spearmanr_res.append(corr)

        corr, _ = kendalltau(bt, bp)
        if not np.isnan(corr):
            kendalltau_res.append(corr)

    print("Spearman:", np.mean(spearmanr_res).round(5))
    print("Kendall:", np.mean(kendalltau_res).round(5))

In [ ]:
# To score whole team we can sum rank of all players in the team to get the team rank
# Then compare teams by this rank

In [285]:
def evaluate_model(model, df_test):
    if isinstance(model, LinearRegression):
        coeffs = model.coef_[0:PLAYERS_COUNT]
    elif isinstance(model, LogisticRegression):
        coeffs = model.coef_[0][0:PLAYERS_COUNT]
    else:
        coeffs = model.w[1:PLAYERS_COUNT+1]
    
    player_to_predicted_rank = {player_idx_to_id[i]:coeff for i, coeff in enumerate(coeffs)}
    
    df_test_known = df_test.copy()[["position", "team_members", "tournament_id", "team_id"]]
    df_test_known["team_members"] = df_test_known["team_members"].apply(
        lambda t: [id for id in t if id in train_players]
    )
    df_test_known = df_test_known[df_test_known["team_members"].apply(lambda t: len(t) > 0)]

    df_test_known["pred_rating"] = df_test_known["team_members"].apply(
        lambda t: -np.sum([player_to_predicted_rank[id] for id in t])
    )
    
    score_result_df(df_test_known)   
    
evaluate_model(lr_model, df_test)

Spearman: 0.7778
Kendall: 0.61366


In [ ]:
# Lets try alternative way to score team, predict how whole team of player will answer all question
# and rate by results

In [284]:
def evaluate_model_via_predict(model, df_test):   
    predicted_rank = []
    
    df_test_known = df_test.copy()[["position", "team_members", "tournament_id", "team_id"]]
    df_test_known["team_members"] = df_test_known["team_members"].apply(
        lambda t: [id for id in t if id in train_players]
    )
    df_test_known = df_test_known[df_test_known["team_members"].apply(lambda t: len(t) > 0)]
    
    for _, row in df_test_known.iterrows():
        x = np.zeros(PLAYERS_COUNT + (TOUR_QUESTIONS_COUNT * len(tournament_ids)))
        idx = player_ids_to_idx_func(row["team_members"])
        x[idx] = 1
        x[PLAYERS_COUNT:] = 1
        rank = model.predict(x.reshape((1, x.shape[0])))
        predicted_rank.append(-rank)
    

    df_test_known["pred_rating"] = predicted_rank
    
    score_result_df(df_test_known)   
    
evaluate_model_via_predict(lr_model, df_test)

Spearman: 0.7778
Kendall: 0.61366


In [ ]:
# Yes, it shows the same result

# 4. EM-algorithm

In [156]:
from scipy.sparse import lil_matrix, csr_matrix, hstack
from scipy.special import expit as sigmoid

In [287]:
em_bootstrap_model = LogisticRegression(tol=1e-1, solver='saga', penalty='none')
em_bootstrap_model.fit(players_train_mtx, train_y)

LogisticRegression(penalty='none', solver='saga', tol=0.1)

In [297]:
class EMLogReg:
    def __init__(self, init_classifier=None):
        self.w = np.hstack((init_classifier.intercept_, init_classifier.coef_[0]))
        
    def coef_(self):
        return self.w
        
    def fit(self, X, y, max_iters=100, lr=1, tol=0.00001):
        X = hstack([np.ones(len(y)).reshape(-1, 1), X], format='csr')        
            
        losses = []
        prev_mean_loss = np.inf
        
        data_len = X.shape[0]
        prev_loss = np.inf
        
        for i in range(max_iters):
            X_batch = X
            y_batch = y
            preds = sigmoid(X_batch.dot(self.w))
            loss = self.log_loss(y_batch, preds)
            
            if (i + 1) % 10 == 0:
                if abs(prev_loss - loss) < tol:
                    break    
                
                prev_loss = loss

            grad = -X_batch.T.dot(y_batch - preds) / len(y_batch)
            self.w -= lr * grad 
            
        print(f"Iteration: {i} Loss: {loss.round(5)} {np.max(grad)}")
    
    def predict_proba(self, X):
        X = hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
        preds = sigmoid(X.dot(self.w))
        return np.hstack(((1 - preds).reshape(-1, 1), preds.reshape(-1, 1)))
    
    def log_loss(self, y, p):
        return -np.mean(y * np.log(p) + (1 - y) * np.log(1 - p))

In [298]:
em_model = EMLogReg(em_bootstrap_model)
evaluate_model(em_model, df_test)

Spearman: 0.77455
Kendall: 0.60901


In [300]:
em_model = EMLogReg(em_bootstrap_model)
print("Bootstrapped model")
evaluate_model(em_model, df_test)
print("")

for step in range(100):
    # E-step
    preds = em_model.predict_proba(players_train_mtx)

    offset = 0
    for members_count in members_count_seq:
        for qi in range(TOUR_QUESTIONS_COUNT):
            team = range(offset, offset + members_count)
            
            team_lose_proba = preds[:, 0][team].prod()
            team_win_proba = 1 - team_lose_proba
            preds[:, 1][team] /= team_win_proba
            
            offset += members_count
            
    z = preds[:, 1].clip(0, 1)
    z[train_y == 0] = 0
    
    # M-step
    em_model.fit(players_train_mtx, z, max_iters=100, tol=0.000001, lr=9)

    print("\nStep:", step + 1)
    evaluate_model(em_model, df_test)
    print("")

Bootstrapped model
Spearman: 0.77455
Kendall: 0.60901

Iteration: 99 Loss: 0.38696 1.4104073555722075e-05

Step: 1
Spearman: 0.77451
Kendall: 0.60901

Iteration: 99 Loss: 0.35825 1.0574944561446e-05

Step: 2
Spearman: 0.77457
Kendall: 0.60911

Iteration: 99 Loss: 0.34388 9.187755076839931e-06

Step: 3
Spearman: 0.77463
Kendall: 0.60916

Iteration: 99 Loss: 0.33667 1.1817410293591478e-05

Step: 4
Spearman: 0.77479
Kendall: 0.60932

Iteration: 99 Loss: 0.33307 1.3247109253623162e-05

Step: 5
Spearman: 0.7748
Kendall: 0.60933

Iteration: 99 Loss: 0.33127 1.4349700301721131e-05

Step: 6
Spearman: 0.77476
Kendall: 0.60929

Iteration: 99 Loss: 0.33036 1.4737887415486674e-05

Step: 7
Spearman: 0.77478
Kendall: 0.60935

Iteration: 99 Loss: 0.32991 1.475968913815003e-05

Step: 8
Spearman: 0.77474
Kendall: 0.60923

Iteration: 99 Loss: 0.32968 1.4599482167863897e-05

Step: 9
Spearman: 0.77475
Kendall: 0.60921

Iteration: 99 Loss: 0.32956 1.435161121039868e-05

Step: 10
Spearman: 0.77486
Kendall: 

Iteration: 99 Loss: 0.32913 5.222728547493622e-06

Step: 86
Spearman: 0.77739
Kendall: 0.61261

Iteration: 99 Loss: 0.32913 5.168506689695889e-06

Step: 87
Spearman: 0.77742
Kendall: 0.61267

Iteration: 99 Loss: 0.32913 5.114767106032538e-06

Step: 88
Spearman: 0.77749
Kendall: 0.61289

Iteration: 99 Loss: 0.32913 5.061507979489016e-06

Step: 89
Spearman: 0.77751
Kendall: 0.61291

Iteration: 99 Loss: 0.32912 5.008727443072272e-06

Step: 90
Spearman: 0.77754
Kendall: 0.61289

Iteration: 99 Loss: 0.32912 4.95642358113128e-06

Step: 91
Spearman: 0.77777
Kendall: 0.61304

Iteration: 99 Loss: 0.32912 4.904594430675876e-06

Step: 92
Spearman: 0.77776
Kendall: 0.61303

Iteration: 99 Loss: 0.32911 4.853237982683727e-06

Step: 93
Spearman: 0.77775
Kendall: 0.61298

Iteration: 99 Loss: 0.32911 4.802352183406914e-06

Step: 94
Spearman: 0.77774
Kendall: 0.61299

Iteration: 99 Loss: 0.32911 4.7519349356641115e-06

Step: 95
Spearman: 0.77788
Kendall: 0.61317

Iteration: 99 Loss: 0.3291 4.70198410013

In [196]:
em_model = EMLogReg(em_bootstrap_model)
print("Bootstrapped model")
evaluate_model(em_model, df_test)
print("")

for step in range(50):
    # E-step
    preds = em_model.predict_proba(players_train_mtx)

    offset = 0
    for members_count in members_count_seq:
        for qi in range(TOUR_QUESTIONS_COUNT):
            team = range(offset, offset + members_count)
            
            team_lose_proba = preds[:, 0][team].prod()
            team_win_proba = 1 - team_lose_proba
            preds[:, 1][team] /= team_win_proba
            
            offset += members_count
            
    z = preds[:, 1].clip(0, 1)
    z[train_y == 0] = 0
    
    # M-step
    em_model.fit(players_train_mtx, z, max_iters=50, tol=0.000001, lr=10)

    print("\nStep:", step + 1)
    evaluate_model(em_model, df_test)
    print("")

Bootstrapped model
Spearman: 0.69333
Kendall: 0.53117

Iteration: 49 Loss: 0.42104

Step: 1
Spearman: 0.69324
Kendall: 0.53106

Iteration: 49 Loss: 0.37347

Step: 2
Spearman: 0.69298
Kendall: 0.53073

Iteration: 49 Loss: 0.34907

Step: 3
Spearman: 0.69297
Kendall: 0.53077

Iteration: 49 Loss: 0.33751

Step: 4
Spearman: 0.69302
Kendall: 0.53086

Iteration: 49 Loss: 0.33224

Step: 5
Spearman: 0.69311
Kendall: 0.5309

Iteration: 49 Loss: 0.32987

Step: 6
Spearman: 0.69337
Kendall: 0.53125

Iteration: 49 Loss: 0.32881

Step: 7
Spearman: 0.69381
Kendall: 0.53147

Iteration: 49 Loss: 0.32832

Step: 8
Spearman: 0.69399
Kendall: 0.53159

Iteration: 49 Loss: 0.32809

Step: 9
Spearman: 0.69407
Kendall: 0.53165

Iteration: 49 Loss: 0.32797

Step: 10
Spearman: 0.69409
Kendall: 0.53167

Iteration: 49 Loss: 0.3279

Step: 11
Spearman: 0.69419
Kendall: 0.53191

Iteration: 49 Loss: 0.32786

Step: 12
Spearman: 0.69432
Kendall: 0.53201

Iteration: 49 Loss: 0.32782

Step: 13
Spearman: 0.69436
Kendall: 0.5

# 5. Questions rating

In [270]:
from itertools import groupby

In [259]:
# Get questions rating from model coeffs, harder question -> lower coeff

question_coeff = lr_model.coef_[PLAYERS_COUNT:]
question_coeff.shape

(14328,)

In [279]:
def calc_question_rating(question_coeff):
    question_rating = [(c, tournament_ids[i // TOUR_QUESTIONS_COUNT]) for i, c in enumerate(question_coeff)]
    question_rating.sort(key=lambda x: x[1])

    res = []

    for tour_id, questions in groupby(question_rating, key=lambda x: x[1]):
        sum_rating = np.sum([r for r, _ in questions])
        res.append((sum_rating, tour_id))

    res.sort()
    return res
    
question_rating = calc_question_rating(question_coeff)
df_tours_view = df_tournaments_norm.drop(columns=["dateStart", "dateEnd", "questionQty"])

In [282]:
# Top hardest tours

hardest_tour_ids = list(map(lambda x: x[1], question_rating[0:20]))
df_tours_view[df_tours_view["id"].isin(hardest_tour_ids)]

,id,name,type,season
4225,5083,Ускользающая сова,Синхрон,52.0
4240,5098,"Ра-II: синхрон ""Борского корабела""",Синхрон,52.0
4299,5159,Первенство правого полушария,Синхрон,52.0
4301,5161,Антибинго,Синхрон,52.0
4325,5186,VERSUS: Коробейников vs. Матвеев,Синхрон,52.0
4545,5412,Серия Premier. Седьмая печать,Синхрон,52.0
4554,5421,Львов зимой. Адвокат,Синхрон,52.0
4631,5501,Кубок Кайнозоя,Синхрон,52.0
4645,5515,Чемпионат Минска. Лига А. Тур четвёртый,Обычный,52.0
4690,5563,Линч,Синхрон,52.0


In [281]:
# Top easiest tours

easiest_tour_ids = list(map(lambda x: x[1], question_rating[-20:]))
df_tours_view[df_tours_view["id"].isin(easiest_tour_ids)]

,id,name,type,season
4150,5008,Школьный Синхрон-lite. Выпуск 2.3,Асинхрон,52.0
4151,5009,(а)Синхрон-lite. Лига старта. Эпизод III,Асинхрон,52.0
4153,5011,(а)Синхрон-lite. Лига старта. Эпизод IV,Асинхрон,52.0
4154,5012,Школьный Синхрон-lite. Выпуск 2.5,Асинхрон,52.0
4155,5013,(а)Синхрон-lite. Лига старта. Эпизод V,Асинхрон,52.0
4570,5438,Синхрон Лиги Разума,Синхрон,52.0
4723,5601,Межфакультетский кубок МГУ. Отбор №4,Обычный,52.0
4812,5697,Школьный Синхрон-lite. Выпуск 3.1,Асинхрон,53.0
4813,5698,(а)Синхрон-lite. Лига старта. Эпизод VII,Асинхрон,53.0
4816,5701,Школьный Синхрон-lite. Выпуск 3.3,Асинхрон,53.0
